In [1]:
import tensorflow as tf



In [66]:
def FCN_model(len_classes=141, dropout_rate=0.2):
    
    input = tf.keras.layers.Input(shape=(1, 12, 1))

    x = tf.keras.layers.Conv2D(filters=4, kernel_size=1, strides=1)(input)
    x = tf.keras.layers.Dropout(dropout_rate)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)

    # x = tf.keras.layers.MaxPooling2D()(x)

    x = tf.keras.layers.Conv2D(filters=8, kernel_size=1, strides=2)(x)
    x = tf.keras.layers.Dropout(dropout_rate)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)

    # x = tf.keras.layers.MaxPooling2D()(x)

    x = tf.keras.layers.Conv2D(filters=16, kernel_size=1, strides=2)(x)
    x = tf.keras.layers.Dropout(dropout_rate)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)

    # x = tf.keras.layers.MaxPooling2D()(x)

    x = tf.keras.layers.Conv2D(filters=32, kernel_size=1, strides=2)(x)
    x = tf.keras.layers.Dropout(dropout_rate)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)

    # x = tf.keras.layers.MaxPooling2D()(x)

    x = tf.keras.layers.Conv2D(filters=64, kernel_size=1, strides=2)(x)
    x = tf.keras.layers.Dropout(dropout_rate)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)

    # Uncomment the below line if you're using dense layers
    # x = tf.keras.layers.GlobalMaxPooling2D()(x)

    # Fully connected layer 1
    # x = tf.keras.layers.Dropout(dropout_rate)(x)
    # x = tf.keras.layers.BatchNormalization()(x)
    # x = tf.keras.layers.Dense(units=64)(x)
    # x = tf.keras.layers.Activation('relu')(x)

    # Fully connected layer 1
    x = tf.keras.layers.Conv2D(filters=8, kernel_size=1, strides=2)(x)
    x = tf.keras.layers.Dropout(dropout_rate)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)

    # Fully connected layer 2
    # x = tf.keras.layers.Dropout(dropout_rate)(x)
    # x = tf.keras.layers.BatchNormalization()(x)
    # x = tf.keras.layers.Dense(units=len_classes)(x)
    # predictions = tf.keras.layers.Activation('softmax')(x)

    # Fully connected layer 2
    x = tf.keras.layers.Conv2D(filters=len_classes, kernel_size=1, strides=2)(x)
    x = tf.keras.layers.Dropout(dropout_rate)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.GlobalMaxPooling2D()(x)
    predictions = tf.keras.layers.Activation('softmax')(x)

    model = tf.keras.Model(inputs=input, outputs=predictions)
    
    print(model.summary())
    print(f'Total number of layers: {len(model.layers)}')

    return model

In [5]:
def train(model, train_generator, val_generator, epochs = 50):
    model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.0001),
                    loss='categorical_crossentropy',
                    metrics=['accuracy'])

    checkpoint_path = './snapshots'
    os.makedirs(checkpoint_path, exist_ok=True)
    model_path = os.path.join(checkpoint_path, 'model_epoch_{epoch:02d}_loss_{loss:.2f}_acc_{acc:.2f}_val_loss_{val_loss:.2f}_val_acc_{val_acc:.2f}.h5')
    
    history = model.fit_generator(generator=train_generator,
                                    steps_per_epoch=len(train_generator),
                                    epochs=epochs,
                                    callbacks=[tf.keras.callbacks.ModelCheckpoint(model_path, monitor='val_loss', save_best_only=True, verbose=1)],
                                    validation_data=val_generator,
                                    validation_steps=len(val_generator))

    return history

In [42]:
import pandas as pd
COL_NAMES_NOTES = ["C", "Cs", "D", "Ds", "E", "F", "Fs", "G", "Gs", "A", "As", "B"]
training_data = pd.read_csv('training_data_full.csv', header=None, names=['idx', "C", "Cs", "D", "Ds", "E", "F", "Fs", "G", "Gs", "A", "As", "B", "start", "end", "chord", "cleaned_chord"])

In [51]:
import numpy as np
len(training_data.cleaned_chord.unique())
msk = np.random.rand(len(training_data)) < 0.8
train = training_data[msk]
test = training_data[~msk]



38752

In [90]:
model = FCN_model(141, 0.2)

Model: "functional_25"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_15 (InputLayer)        [(None, 1, 12, 1)]        0         
_________________________________________________________________
conv2d_84 (Conv2D)           (None, 1, 12, 4)          8         
_________________________________________________________________
dropout_82 (Dropout)         (None, 1, 12, 4)          0         
_________________________________________________________________
batch_normalization_82 (Batc (None, 1, 12, 4)          16        
_________________________________________________________________
activation_83 (Activation)   (None, 1, 12, 4)          0         
_________________________________________________________________
conv2d_85 (Conv2D)           (None, 1, 6, 8)           40        
_________________________________________________________________
dropout_83 (Dropout)         (None, 1, 6, 8)         

In [91]:
model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.0001),
                    loss='categorical_crossentropy',
                    metrics=['accuracy'])

In [88]:
from sklearn import preprocessing

train_chords = train[COL_NAMES_NOTES]
train_labels = train['cleaned_chord']

classes = training_data['cleaned_chord']
lb = preprocessing.LabelBinarizer()
lb.fit(classes)

train_labels = np.array(lb.transform(train_labels), dtype='float32')

print(train_chords.shape)
train_chords = train_chords.values.reshape(-1,1,12,1)
print(train_chords.shape)
print(train_labels.shape)


(154314, 12)
(154314, 1, 12, 1)
(154314, 141)


In [93]:


model.fit(train_chords, train_labels, epochs=50)

Epoch 1/50
4823/4823 [==============================] - 18s 4ms/step - loss: 3.3769 - accuracy: 0.1570
Epoch 2/50
4823/4823 [==============================] - 19s 4ms/step - loss: 3.3700 - accuracy: 0.1567
Epoch 3/50
4823/4823 [==============================] - 20s 4ms/step - loss: 3.3665 - accuracy: 0.1566
Epoch 4/50
4823/4823 [==============================] - 20s 4ms/step - loss: 3.3620 - accuracy: 0.1564
Epoch 5/50
4823/4823 [==============================] - 20s 4ms/step - loss: 3.3606 - accuracy: 0.1571
Epoch 6/50
4823/4823 [==============================] - 20s 4ms/step - loss: 3.3579 - accuracy: 0.1573
Epoch 7/50
4823/4823 [==============================] - 20s 4ms/step - loss: 3.3569 - accuracy: 0.1573
Epoch 8/50
4823/4823 [==============================] - 20s 4ms/step - loss: 3.3551 - accuracy: 0.1570
Epoch 9/50
4823/4823 [==============================] - 20s 4ms/step - loss: 3.3539 - accuracy: 0.1570
Epoch 10/50
4823/4823 [==============================] - 20s 4ms/step - l